# Run the `HeatDiffusion` model through its BMI

`HeatDiffusion` models the diffusion of temperature on a uniform rectangular plate with Dirichlet boundary conditions.
Wrapping `HeatDiffusion` with a [Basic Model Interface](https://bmi.readthedocs.io) (BMI),
lets you you control the model through a standard set of functions,
so you don't have to know the details of how the model is run.

## Setup

[NetLogo](https://ccl.northwestern.edu/netlogo/) is required to run this notebook.
Once you've [downloaded and installed](https://ccl.northwestern.edu/netlogo/bind/article/getting-started-with-netlogo.html) NetLogo,
edit the configuration file in the `examples` directory
to set the path to the install location in the *netlogo_home* field.

View the configuration file.

In [ ]:
cat "config.yaml"

The current value of *netlogo_home* is where NetLogo is installed on the [EarthscapeHub](https://csdms.colorado.edu/wiki/JupyterHub) *lab* and *jupyter* Hubs.

Import the Python libraries we'll use below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Initialize the model

Import the `HeatDiffusion` BMI.

In [ ]:
from heat import BmiHeatDiffusion

Make an instance of the model through its BMI.

In [ ]:
m = BmiHeatDiffusion()

Get the name of the model.

In [ ]:
m.get_component_name()

Initialize the model using parameter values from the configuration file.

In [ ]:
m.initialize("config.yaml")

## Get model information

List the model's input and output variables (also called "exchange items").

In [ ]:
print("Number of input variables:", m.get_input_item_count())
for var in m.get_input_var_names():
    print(f" - {var}")

In [ ]:
print("Number of output variables:", m.get_output_item_count())
for var in m.get_output_var_names():
    print(f" - {var}")

The BMI exposes one output variable, `plate_surface__temperature`, that maps to the `temperature` variable in the Netlogo `HeatDiffusion` model.
The long variable name is an example of a CSDMS [Standard Name](https://csdms.colorado.edu/wiki/CSDMS_Standard_Names).

Get more information on the `plate_surface__temperature` variable.

In [ ]:
var_name = m.get_output_var_names()[0]
print(f"Variable {var_name}")
print(" - type:", m.get_var_type(var_name))
print(" - units:", m.get_var_units(var_name))
print(" - itemsize:", m.get_var_itemsize(var_name))
print(" - nbytes:", m.get_var_nbytes(var_name))
print(" - location:", m.get_var_location(var_name))

In a BMI, all variables are defined on grids.

Get information about the grid used by the `plate_surface__temperature` variable.

In [ ]:
grid_id = m.get_var_grid(var_name)
print(" - grid id:", grid_id)
print(" - grid type:", m.get_grid_type(grid_id))
grid_rank = m.get_grid_rank(grid_id)
print(" - rank:", grid_rank)
grid_size = m.get_grid_size(grid_id)
print(" - size:", grid_size)
grid_shape = np.empty(grid_rank, dtype=np.int32)
m.get_grid_shape(grid_id, grid_shape)
print(" - shape:", grid_shape)
grid_spacing = np.empty(grid_rank, dtype=np.float64)
m.get_grid_spacing(grid_id, grid_spacing)
print(" - spacing:", grid_spacing)
grid_origin = np.empty(grid_rank, dtype=np.float64)
m.get_grid_origin(grid_id, grid_origin)
print(" - origin:", grid_origin)

Get time information from the model.

In [ ]:
print("Start time:", m.get_start_time())
print("End time:", m.get_end_time())
print("Current time:", m.get_current_time())
print("Time step:", m.get_time_step())
print("Time units:", m.get_time_units())

## View initial model state

Get the initial temperature distribution on the plate.

In [ ]:
val = np.empty(grid_size, dtype=m.get_var_type(var_name))
m.get_value(var_name, val)
print(f"Temperature at time {m.get_current_time()}:")
val

Note that the temperatures are returned as a one-dimensional NumPy array.

As a metric, report the sum of the temperature values on the plate.

In [ ]:
val.sum()

Visualize the temperature values using the *plot_temperature* helper function.
Note that the temperature array has to be redimensionalized.

In [ ]:
from helpers import plot_temperature

plot_temperature(val.reshape(grid_shape), plate_size=m.get_attribute("plate-size"))

## Run the model

Advance the model one time step.

In [ ]:
m.update()
print(f"Time: {m.get_current_time()} {m.get_time_units()}")

Have the temperature values changed?

In [ ]:
m.get_value(var_name, val)
print(f"Temperature at time {m.get_current_time()}:", val)
print(f"Sum: {val.sum()} {m.get_var_units(var_name)}")

The sum shows that the plate has cooled slightly.

Run the model to some distant time.

In [ ]:
distant_time = 20.0
while m.get_current_time() < distant_time:
    m.update()
print(f"Time: {m.get_current_time()} {m.get_time_units()}")

How has the temperature field has evolved?

In [ ]:
m.get_value(var_name, val)
print(f"Temperature at time {m.get_current_time()}:", val)
print(f"Sum: {val.sum()} {m.get_var_units(var_name)}")

Visualize the current temperature field.

In [ ]:
plot_temperature(val.reshape(grid_shape), plate_size=m.get_attribute("plate-size"))

Diffusion!

## Finalize the model

Shut down the model when we're finished.

In [ ]:
m.finalize()